In [1]:
import pandas as pd
import re
import numpy as np
from sklearn.model_selection import train_test_split
from collections import Counter
import sentencepiece as spm
import os
# Read the dataset
df = pd.read_csv('final_main_dataset.tsv', sep='\t', encoding='utf-8')

print(f"Dataset loaded successfully!")

# Breif Eda on Dataset
print(f"Total rows: {len(df)}")
print(f"Columns: {df.columns.tolist()}")
print("\nFirst few sentences:")
print(df['sentence'].head())

def normalize_urdu_text(text):

    if pd.isna(text):  # Handle NaN values
        return ""

    text = str(text)

    # remove common urdu diacritics ie standardize alef and yeh forms(zabar, zer, pesh, etc.)
    # Some most used and common standardized
    diacritics = [
        '\u064B',
        '\u064C',
        '\u064D',
        '\u064E',
        '\u064F',
        '\u0650',
        '\u0651',
        '\u0652',
        '\u0653',
        '\u0654',
        '\u0655',
        '\u0656',
        '\u0657',
        '\u0658',
    ]

    for diacritic in diacritics:
        text = text.replace(diacritic, '')

    # Standardize different forms of alef
    text = text.replace('أ', 'ا')
    text = text.replace('إ', 'ا')
    text = text.replace('آ', 'ا')
    text = text.replace('ٱ', 'ا')

    # Standardize different forms of yeh
    text = text.replace('ی', 'ی')
    text = text.replace('ے', 'ے')
    text = text.replace('ئ', 'ی')

    # Standardize heh forms
    text = text.replace('ۃ', 'ہ')
    text = text.replace('ھ', 'ہ')

    # Remove extra whitespaces
    text = ' '.join(text.split())

    return text.strip()

# Test the function
test_sentence = "کبھی کبھار ہی خیالی پلاو بناتا ہوں"
normalized = normalize_urdu_text(test_sentence)
print(f"Original:   {test_sentence}")
print(f"Normalized: {normalized}")
print("\nNormalization function ready!")


# Apply Normalization to All Sentences

df['normalized_sentence'] = df['sentence'].apply(normalize_urdu_text)

# Remove empty sentences after normalization
df = df[df['normalized_sentence'].str.len() > 0].reset_index(drop=True)

print(f"Normalization complete!")

print(f"Total sentences after cleaning: {len(df)}")
print("\nSample normalized sentences:")
for i in range(5):
    print(f"{i+1}. {df['normalized_sentence'].iloc[i]}")


# Save Normalized Sentences to Text File for tokenizer training

with open('urdu_sentences.txt', 'w', encoding='utf-8') as f:
    for sentence in df['normalized_sentence']:
        f.write(sentence + '\n')
print(f"Saved {len(df)} normalized sentences to 'urdu_sentences.txt'")

# Train SentencePiece Tokenizer with Automatic Vocab Size

print("Training SentencePiece Unigram tokenizer...")

# Checking unique characters
with open('urdu_sentences.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    unique_chars = len(set(text))

print(f"Unique characters in dataset: {unique_chars}")

# Set Reasonable vocab size based on dataset size
total_sentences = len(df)

if total_sentences < 5000:
    vocab_size = 3000
elif total_sentences < 10000:
    vocab_size = 4000
elif total_sentences < 15000:
    vocab_size = 5000
else:
    vocab_size = 6000

print(f"Setting vocabulary size to: {vocab_size}")
print("Training tokenizer...\n")

# Train the tokenizer with adjusted vocab size
spm.SentencePieceTrainer.train(
    input='urdu_sentences.txt',
    model_prefix='urdu_tokenizer',
    vocab_size=vocab_size,
    character_coverage=1.0,  # Cover all Urdu characters
    model_type='unigram',  # Unigram Language Model
    pad_id=0,  # Padding token
    unk_id=1,  # Unknown token
    bos_id=2,  # Begin of sentence
    eos_id=3,  # End of sentence
    user_defined_symbols=['[MASK]'],  # Masking special tokens
    num_threads=4,
)

print("Tokenizer trained successfully!")
print(f"Vocabulary size: {vocab_size}")
print("Files created: urdu_tokenizer.model, urdu_tokenizer.vocab")


# Load the trained tokenizer
sp = spm.SentencePieceProcessor()
sp.load('urdu_tokenizer.model')
print(f"Vocabulary size: {sp.vocab_size()}")
print(f"\n--- Testing Tokenizer ---")

# Test sentences
test_sentences = [
    "کبھی کبھار ہی خیالی پلاو بناتا ہوں",
    "آج موسم بہت اچھا ہے",
    "بولو کیسے ہو"
]

for sent in test_sentences:
    tokens = sp.encode_as_pieces(sent)
    ids = sp.encode_as_ids(sent)

    print(f"\nOriginal:  {sent}")
    print(f"Tokens:    {tokens}")
    print(f"Token IDs: {ids[:10]}...")  # Show first 10 IDs

    # Decode back
    decoded = sp.decode_pieces(tokens)
    print(f"Decoded:   {decoded}")

print("\nTokenizer working correctly!")

# Tokenize All Sentences

print("Tokenizing all sentences...")

# Tokenize all normalized sentences
df['tokenized'] = df['normalized_sentence'].apply(lambda x: sp.encode_as_ids(x))
df['token_count'] = df['tokenized'].apply(len)

print(f" All sentences tokenized!")
print(f"\nToken Statistics")
print(f"Average tokens per sentence: {df['token_count'].mean():.2f}")
print(f"Min tokens: {df['token_count'].min()}")
print(f"Max tokens: {df['token_count'].max()}")
print(f"Median tokens: {df['token_count'].median():.2f}")

# Show distribution
print(f"\nToken Length Distribution ")
print(df['token_count'].describe())

#  Split Dataset (80% Train, 10% Val, 10% Test)

print("Splitting dataset...")

# First split: 80% train, 20% temp
train_df, temp_df = train_test_split(
    df,
    test_size=0.2,
    random_state=42,
    shuffle=True
)

# Second split: 10% val, 10% test (from the 20% temp)
val_df, test_df = train_test_split(
    temp_df,
    test_size=0.5,
    random_state=42,
    shuffle=True
)

# Reset indices
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

print(f" Dataset split complete!")
print(f"\nSplit Statistics")
print(f"Training set:   {len(train_df)} sentences ({len(train_df)/len(df)*100:.1f}%)")
print(f"Validation set: {len(val_df)} sentences ({len(val_df)/len(df)*100:.1f}%)")
print(f"Test set:       {len(test_df)} sentences ({len(test_df)/len(df)*100:.1f}%)")
print(f"Total:          {len(df)} sentences")

#  Save Processed Datasets

print("Saving processed datasets...")

# Save to CSV files
train_df.to_csv('train_data.csv', index=False)
val_df.to_csv('val_data.csv', index=False)
test_df.to_csv('test_data.csv', index=False)

print(" Saved datasets:")
print("  train_data.csv")
print("  val_data.csv")
print("  test_data.csv")

# Save tokenized versions for further use
import pickle

with open('train_tokenized.pkl', 'wb') as f:
    pickle.dump(train_df['tokenized'].tolist(), f)

with open('val_tokenized.pkl', 'wb') as f:
    pickle.dump(val_df['tokenized'].tolist(), f)

with open('test_tokenized.pkl', 'wb') as f:
    pickle.dump(test_df['tokenized'].tolist(), f)

print("Saved tokenized versions (pickle files)")


# Summary and Verification

print("PREPROCESSING COMPLETE - SUMMARY")


print(f"\nDataset Statistics:")
print(f"   Total sentences: {len(df)}")
print(f"   Train: {len(train_df)} | Val: {len(val_df)} | Test: {len(test_df)}")

print(f"\nTokenizer:")
print(f"   Vocabulary size: {sp.vocab_size()}")
print(f"   Model type: Unigram LM")
print(f"   Special tokens: <pad>, <unk>, <s>, </s>, [MASK]")

print(f"\n Files Created:")
files = [
    'urdu_tokenizer.model',
    'urdu_tokenizer.vocab',
    'train_data.csv',
    'val_data.csv',
    'test_data.csv',
    'train_tokenized.pkl',
    'val_tokenized.pkl',
    'test_tokenized.pkl'
]

for file in files:
    exists = "✓" if os.path.exists(file) else "✗"
    print(f"   {exists} {file}")

print(f"\n Sample from Training Set:")
sample = train_df.sample(3)
for idx, row in sample.iterrows():
    print(f"\n   Sentence: {row['normalized_sentence']}")
    print(f"   Tokens: {sp.encode_as_pieces(row['normalized_sentence'])[:8]}...")
    print(f"   Token count: {row['token_count']}")

print("\Phase 1 Preprocessing Complete!")

<>:288: SyntaxWarning: invalid escape sequence '\P'
<>:288: SyntaxWarning: invalid escape sequence '\P'
C:\Users\G T\AppData\Local\Temp\ipykernel_9004\2005792912.py:288: SyntaxWarning: invalid escape sequence '\P'
  print("\Phase 1 Preprocessing Complete!")


Dataset loaded successfully!
Total rows: 20000
Columns: ['client_id', 'path', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accents', 'variant', 'locale', 'segment']

First few sentences:
0                   کبھی کبھار ہی خیالی پلاو بناتا ہوں
1                    اور پھر ممکن ہے کہ پاکستان بھی ہو
2                        یہ فیصلہ بھی گزشتہ دو سال میں
3                       ان کے بلے بازوں کے سامنے ہو گا
4    آبی جانور میں بطخ بگلا اور دُوسْرا آبی پرندہ ش...
Name: sentence, dtype: object
Original:   کبھی کبھار ہی خیالی پلاو بناتا ہوں
Normalized: کبہی کبہار ہی خیالی پلاو بناتا ہوں

Normalization function ready!
Normalization complete!
Total sentences after cleaning: 20000

Sample normalized sentences:
1. کبہی کبہار ہی خیالی پلاو بناتا ہوں
2. اور پہر ممکن ہے کہ پاکستان بہی ہو
3. یہ فیصلہ بہی گزشتہ دو سال میں
4. ان کے بلے بازوں کے سامنے ہو گا
5. ابی جانور میں بطخ بگلا اور دوسرا ابی پرندہ شامل ہونا
Saved 20000 normalized sentences to 'urdu_sentences.txt'
Training SentencePiece Un

In [2]:
import torch
import torch.nn as nn
import math

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=512):
        super().__init__()

        # Create positional encoding matrix
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)  # Add batch dimension
        self.register_buffer('pe', pe)

    def forward(self, x):
        # x shape: (batch_size, seq_len, d_model)
        return x + self.pe[:, :x.size(1), :]

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        assert d_model % num_heads == 0

        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads # it is the dimension of each head key, query and value vectors

        # Each token embedding divided into query, key and value vectors
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)

    def forward(self, query, key, value, mask=None):
        batch_size = query.size(0)

        # Linear projections
        Q = self.W_q(query)
        K = self.W_k(key)
        V = self.W_v(value)

       # Project and split into heads: (B, L, d_model) -> (B, h, L, d_head)
        Q = Q.view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        K = K.view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        V = V.view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)

        # compute attention scores b/w every token pair
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k) # divide by sqrt(d_k) to prevent large values

        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)

        attn = torch.softmax(scores, dim=-1)
        output = torch.matmul(attn, V)
         # Concatenate heads: (B, h, L, d_head) -> (B, L, d_model)
        output = output.transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)
        return self.W_o(output)

class FeedForward(nn.Module):   # after applying attention doing some non-linear transformation to learn more contextual patterns
    def __init__(self, d_model, d_ff=1024, dropout=0.1):
        super().__init__()
        self.linear1 = nn.Linear(d_model, d_ff)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(d_ff, d_model)

    def forward(self, x):
        return self.linear2(self.dropout(torch.relu(self.linear1(x))))

class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super().__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = FeedForward(d_model, d_ff, dropout)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask=None):
        #  Apply Self-attention
        attn_output = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))

        # Apply Feed-forward
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))

        return x

class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super().__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = FeedForward(d_model, d_ff, dropout)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, enc_output, src_mask=None, tgt_mask=None):
        # Self-attention
        attn_output = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))

        # Cross-attention with encoder output
        attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)
        x = self.norm2(x + self.dropout(attn_output))

        # Feed-forward
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ff_output))

        return x

class Transformer(nn.Module):
    def __init__(self, vocab_size, d_model=256, num_heads=2,
                 num_encoder_layers=2, num_decoder_layers=2,
                 d_ff=1024, dropout=0.1, max_len=512):
        super().__init__()

        self.d_model = d_model

        # Add Embeddings and positional encoding
        self.encoder_embedding = nn.Embedding(vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_len)

        # Encoder layers
        self.encoder_layers = nn.ModuleList([
            EncoderLayer(d_model, num_heads, d_ff, dropout)
            for _ in range(num_encoder_layers)
        ])

        # Decoder layers
        self.decoder_layers = nn.ModuleList([
            DecoderLayer(d_model, num_heads, d_ff, dropout)
            for _ in range(num_decoder_layers)
        ])

        # Output projection by mapping decoder output back to vocab size
        self.fc_out = nn.Linear(d_model, vocab_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src, tgt, src_mask=None, tgt_mask=None):
        # Encode
        src_emb = self.dropout(self.positional_encoding(
            self.encoder_embedding(src) * math.sqrt(self.d_model)))

        enc_output = src_emb
        for layer in self.encoder_layers:
            enc_output = layer(enc_output, src_mask)

        # Decode
        tgt_emb = self.dropout(self.positional_encoding(
            self.decoder_embedding(tgt) * math.sqrt(self.d_model)))

        dec_output = tgt_emb
        for layer in self.decoder_layers:
            dec_output = layer(dec_output, enc_output, src_mask, tgt_mask)

        # Output
        output = self.fc_out(dec_output)

        return output

In [3]:
import sentencepiece as spm
sp = spm.SentencePieceProcessor()
sp.load('urdu_tokenizer.model')

vocab_size = sp.vocab_size()
# Create the model
model = Transformer(
    vocab_size=vocab_size,
    d_model=256,
    num_heads=2,
    num_encoder_layers=2,
    num_decoder_layers=2,
    d_ff=1024,
    dropout=0.1,
    max_len=512
)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
# Count total parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Model initialized!")
print(f"Device: {device}")
print(f"Vocabulary size: {vocab_size}")
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")
# Making a dummy input
batch_size = 2
src_len = 10
tgt_len = 8

src = torch.randint(0, vocab_size, (batch_size, src_len)).to(device)
tgt = torch.randint(0, vocab_size, (batch_size, tgt_len)).to(device)

# Forward pass
with torch.no_grad():
    output = model(src, tgt)
print("Model forward pass successful!")
print(f"Input shape (src): {src.shape}")
print(f"Input shape (tgt): {tgt.shape}")
print(f"Output shape: {output.shape}")
print(f"Expected: ({batch_size}, {tgt_len}, {vocab_size})")


Model initialized!
Device: cpu
Vocabulary size: 6000
Total parameters: 8,300,400
Trainable parameters: 8,300,400
Model forward pass successful!
Input shape (src): torch.Size([2, 10])
Input shape (tgt): torch.Size([2, 8])
Output shape: torch.Size([2, 8, 6000])
Expected: (2, 8, 6000)


In [4]:
# Span Corruption + Masked Data Creation
import random
import torch
from torch.utils.data import Dataset, DataLoader

MASK_TOKEN = "[MASK]"
PAD_TOKEN = "[PAD]"
UNK_TOKEN = "[UNK]"
SOS_TOKEN = "<sos>"
EOS_TOKEN = "<eos>"

MASK_PROB = 0.15  # 15% tokens will be masked
RANDOM_TOKEN_PROB = 0.10  # 10% of masked tokens replaced with random token
UNCHANGED_PROB = 0.10      # 10% of masked tokens stay same


def apply_span_corruption(tokens, vocab, mask_prob=MASK_PROB):

   # Apply T5-style span corruption + BERT-style 15% masking rule.
    output_tokens = tokens.copy()
    target_tokens = []
    num_to_mask = max(1, int(len(tokens) * mask_prob)) # number of tokens to mask (atleast 1)

    # Randomly select spans (each span length 1–3)
    mask_indices = set()
    i = 0
    while len(mask_indices) < num_to_mask and i < len(tokens):
        span_len = random.choice([1, 2, 3])
        start = random.randint(0, len(tokens) - span_len)
        for j in range(start, min(start + span_len, len(tokens))):
            mask_indices.add(j)
        i += 1

    # Apply 80/10/10 masking rule
    for idx in range(len(tokens)):
        if idx in mask_indices:
            rand = random.random()
            if rand < 0.8:
                output_tokens[idx] = MASK_TOKEN
            elif rand < 0.9:
                output_tokens[idx] = random.choice(list(vocab.keys()))
            else:
                pass  # keep unchanged
            target_tokens.append(tokens[idx])
        else:
            target_tokens.append(PAD_TOKEN)

    return output_tokens, target_tokens


class MaskedUrduDataset(Dataset):
    def __init__(self, df, vocab, tokenizer, max_len=50):
        self.df = df
        self.vocab = vocab
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        src_text = str(self.df.iloc[idx]['input_text'])
        tgt_text = str(self.df.iloc[idx]['target_text'])

        # Tokenize input (apply corruption)
        tokens = self.tokenizer(src_text)
        corrupted, target = apply_span_corruption(tokens, self.vocab)

        # Convert to IDs
        src_ids = [self.vocab.get(tok, self.vocab[UNK_TOKEN]) for tok in corrupted]
        tgt_ids = [self.vocab.get(tok, self.vocab[UNK_TOKEN]) for tok in target]

        # Add SOS and EOS
        src_ids = [self.vocab[SOS_TOKEN]] + src_ids[:self.max_len-2] + [self.vocab[EOS_TOKEN]]
        tgt_ids = [self.vocab[SOS_TOKEN]] + tgt_ids[:self.max_len-2] + [self.vocab[EOS_TOKEN]]

        # Pad
        src_ids += [self.vocab[PAD_TOKEN]] * (self.max_len - len(src_ids))
        tgt_ids += [self.vocab[PAD_TOKEN]] * (self.max_len - len(tgt_ids))

        return torch.tensor(src_ids), torch.tensor(tgt_ids)


In [6]:
import torch
import torch.nn as nn
import pandas as pd
import pickle
import sentencepiece as spm
import numpy as np
import random
import time
from torch.utils.data import Dataset, DataLoader

# ----------------------------
# Load tokenizer
# ----------------------------
sp = spm.SentencePieceProcessor()
sp.load("urdu_tokenizer.model")

# ----------------------------
# Load pre-tokenized data
# ----------------------------
with open("train_tokenized.pkl", "rb") as f:
    train_data = pickle.load(f)
with open("val_tokenized.pkl", "rb") as f:
    val_data = pickle.load(f)
with open("test_tokenized.pkl", "rb") as f:
    test_data = pickle.load(f)

print("Data loaded successfully!")
print(f"Train: {len(train_data)}, Val: {len(val_data)}, Test: {len(test_data)}")

# ----------------------------
# Span corruption configuration
# ----------------------------
MASK_ID = sp.piece_to_id("<mask>") if sp.piece_to_id("<mask>") != 0 else 1
PAD_ID = 0
SOS_ID = 2
EOS_ID = 3

def apply_span_corruption(token_ids, mask_prob=0.15):
    output_tokens = token_ids.copy()
    target_tokens = token_ids.copy()
    num_to_mask = max(1, int(len(token_ids) * mask_prob))

    mask_indices = set()
    while len(mask_indices) < num_to_mask:
        span_len = random.choice([1, 2, 3])
        start = random.randint(0, len(token_ids) - span_len)
        for j in range(start, min(start + span_len, len(token_ids))):
            mask_indices.add(j)
        if len(mask_indices) >= num_to_mask:
            break

    for idx in range(len(token_ids)):
        if idx in mask_indices:
            r = random.random()
            if r < 0.8:
                output_tokens[idx] = MASK_ID      # 80% mask
            elif r < 0.9:
                output_tokens[idx] = random.randint(4, sp.get_piece_size() - 1)  # 10% random
            # 10% unchanged

    return output_tokens, target_tokens

# ----------------------------
# Masked dataset
# ----------------------------
class MaskedUrduDataset(Dataset):
    def __init__(self, tokenized_sentences, max_len=50):
        self.data = []
        self.max_len = max_len

        for tokens in tokenized_sentences:
            if len(tokens) < 3:
                continue

            corrupted, target = apply_span_corruption(tokens)

            src = [SOS_ID] + corrupted[:max_len-2] + [EOS_ID]
            tgt = [SOS_ID] + target[:max_len-2] + [EOS_ID]

            src += [PAD_ID] * (max_len - len(src))
            tgt += [PAD_ID] * (max_len - len(tgt))

            self.data.append((src, tgt))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return torch.tensor(self.data[idx][0]), torch.tensor(self.data[idx][1])


train_dataset = MaskedUrduDataset(train_data)
val_dataset = MaskedUrduDataset(val_data)
test_dataset = MaskedUrduDataset(test_data)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print("Masked Datasets ready!")
print(f"Train: {len(train_dataset)} | Val: {len(val_dataset)} | Test: {len(test_dataset)}")

# ----------------------------
# Masks for Transformer
# ----------------------------
def create_masks(src, tgt):
    src_mask = (src != PAD_ID).unsqueeze(1).unsqueeze(2)
    seq_len = tgt.size(1)
    tgt_mask = torch.tril(torch.ones((1, seq_len, seq_len), device=tgt.device)).bool()
    return src_mask, tgt_mask

# ----------------------------
# Model + Training Setup
# ----------------------------

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vocab_size = sp.get_piece_size()

# Use the Transformer class already defined in Cell 2
model = Transformer(vocab_size=vocab_size)
model = model.to(device)


criterion = nn.CrossEntropyLoss(ignore_index=PAD_ID)
optimizer = torch.optim.Adam(model.parameters(), lr=2e-4)

num_epochs = 25
best_val_loss = float("inf")

print(f"Training setup complete on {device}")
print("⏭ Training skipped. Loading pretrained model instead...")

# ----------------------------
# Load pretrained model
# ----------------------------
model.load_state_dict(torch.load("best_masked_model.pt", map_location=device))
model.eval()

print("Model loaded successfully and ready for inference!")


Data loaded successfully!
Train: 16000, Val: 2000, Test: 2000
Masked Datasets ready!
Train: 15713 | Val: 1966 | Test: 1981
Training setup complete on cpu
⏭ Training skipped. Loading pretrained model instead...
Model loaded successfully and ready for inference!


In [7]:
import torch
import torch.nn as nn
import pickle
import numpy as np
from torch.utils.data import Dataset, DataLoader
from rouge_score import rouge_scorer
import sacrebleu
import json

PAD_ID = 0
SOS_ID = 2
EOS_ID = 3

# ----------------------------
# Dataset for next-sentence prediction
# ----------------------------
class UrduDataset(Dataset):
    def __init__(self, tokenized_sentences, max_len=50):
        self.data = []
        self.max_len = max_len

        for i in range(len(tokenized_sentences) - 1):
            src_tokens = tokenized_sentences[i]
            tgt_tokens = tokenized_sentences[i + 1]

            if len(src_tokens) < 3 or len(tgt_tokens) < 3:
                continue

            src = [SOS_ID] + src_tokens + [EOS_ID]
            tgt = [SOS_ID] + tgt_tokens + [EOS_ID]

            if len(src) <= max_len and len(tgt) <= max_len:
                self.data.append((src, tgt))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]


def collate_fn(batch):
    src_batch = [b[0] for b in batch]
    tgt_batch = [b[1] for b in batch]

    max_src = max(len(s) for s in src_batch)
    max_tgt = max(len(t) for t in tgt_batch)

    src_padded = [s + [PAD_ID] * (max_src - len(s)) for s in src_batch]
    tgt_padded = [t + [PAD_ID] * (max_tgt - len(t)) for t in tgt_batch]

    return torch.LongTensor(src_padded), torch.LongTensor(tgt_padded)


# ----------------------------
# Load test set
# ----------------------------
with open('test_tokenized.pkl', 'rb') as f:
    test_data = pickle.load(f)

test_dataset = UrduDataset(test_data, max_len=50)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)

print(f"Test data loaded — Samples: {len(test_dataset)}")


# ----------------------------
# Evaluation Function
# ----------------------------
def evaluate_model(model, dataloader, sp, device):
    model.eval()
    references, hypotheses = [], []

    total_loss = 0
    criterion = nn.CrossEntropyLoss(ignore_index=PAD_ID)

    with torch.no_grad():
        for src, tgt in dataloader:
            src, tgt = src.to(device), tgt.to(device)

            tgt_input = tgt[:, :-1]
            tgt_output = tgt[:, 1:]

            src_mask, tgt_mask = create_masks(src, tgt_input)

            output = model(src, tgt_input, src_mask, tgt_mask)

            loss = criterion(output.reshape(-1, output.size(-1)), tgt_output.reshape(-1))
            total_loss += loss.item()

            # ----------------------------
            # Greedy decoding
            # ----------------------------
            for i in range(src.size(0)):
                src_single = src[i:i+1]
                generated = [SOS_ID]

                for _ in range(50):  # max length
                    tgt_tensor = torch.LongTensor([generated]).to(device)

                    src_mask, tgt_mask = create_masks(src_single, tgt_tensor)
                    out = model(src_single, tgt_tensor, src_mask, tgt_mask)

                    next_tok = out[0, -1, :].argmax().item()

                    if next_tok == EOS_ID or next_tok == PAD_ID:
                        break

                    generated.append(next_tok)

                # Decode text
                hypothesis = sp.decode_ids(generated[1:])
                reference_tokens = [t for t in tgt[i].cpu().tolist() if t not in [PAD_ID, SOS_ID, EOS_ID]]
                reference = sp.decode_ids(reference_tokens)

                hypotheses.append(hypothesis)
                references.append([reference])  # sacrebleu format

    print("\nCalculating Metrics...")

    # BLEU & chrF
    bleu = sacrebleu.corpus_bleu(hypotheses, references)
    chrf = sacrebleu.corpus_chrf(hypotheses, references)

    # ROUGE-L
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=False)
    rouge_vals = [
        scorer.score(ref[0], hyp)['rougeL'].fmeasure
        for hyp, ref in zip(hypotheses, references)
    ]
    rouge_l = np.mean(rouge_vals) * 100

    avg_loss = total_loss / len(dataloader)
    perplexity = np.exp(avg_loss)

    return {
        'bleu': bleu.score,
        'rouge_l': rouge_l,
        'chrf': chrf.score,
        'perplexity': perplexity,
        'hypotheses': hypotheses[:5],
        'references': [r[0] for r in references[:5]]
    }


# ----------------------------
# Run Evaluation
# ----------------------------
results = evaluate_model(model, test_loader, sp, device)

print("\n=== EVALUATION RESULTS ===")
print(f"BLEU Score:     {results['bleu']:.2f}")
print(f"ROUGE-L Score:  {results['rouge_l']:.2f}")
print(f"chrF Score:     {results['chrf']:.2f}")
print(f"Perplexity:     {results['perplexity']:.2f}")

print("\n=== QUALITATIVE EXAMPLES ===")
for i in range(len(results['hypotheses'])):
    print(f"\nExample {i+1}")
    print(f"Reference: {results['references'][i]}")
    print(f"Generated: {results['hypotheses'][i]}")
    print("-" * 60)

# ----------------------------
# Save results
# ----------------------------
to_save = {
    'bleu': results['bleu'],
    'rouge_l': results['rouge_l'],
    'chrf': results['chrf'],
    'perplexity': results['perplexity'],
    'examples': [
        {'reference': ref, 'generated': hyp}
        for ref, hyp in zip(results['references'], results['hypotheses'])
    ]
}

with open('evaluation_results.json', 'w', encoding='utf-8') as f:
    json.dump(to_save, f, ensure_ascii=False, indent=2)

print("Results saved to evaluation_results.json")


Test data loaded — Samples: 1961

Calculating Metrics...

=== EVALUATION RESULTS ===
BLEU Score:     16.04
ROUGE-L Score:  0.00
chrF Score:     26.24
Perplexity:     12302.73

=== QUALITATIVE EXAMPLES ===

Example 1
Reference: اس جمود کو توڑا ہے۔
Generated: ہر چیز انتہایی زبردست اور اعلی معیار کی تہی
------------------------------------------------------------

Example 2
Reference: انہوںنے ناریل کے چہلکے سے کاربن حاصل کرکے
Generated: تو اس کو جمود ہے۔
------------------------------------------------------------

Example 3
Reference: اسے پتہ بہی ہے کہ وہ جہوٹ بول رہا ہے
Generated: انہوںنے ناریل کے چہلکے سے کاربن حاصل کرکے
------------------------------------------------------------

Example 4
Reference: افسوس، یہ ہے۔
Generated: اسے وہ بہی بے حسی دیا ہے کہ رہا ہے
------------------------------------------------------------

Example 5
Reference: اوراس کے ساتہ اداروں میں تصادم کو بہی روکنا ہے۔
Generated: یہ افسوس، یہ ہے۔
------------------------------------------------------------
Results 

In [10]:
import streamlit as st
import torch
import torch.nn.functional as F
import sentencepiece as spm

# -------------------------------------------------------
# Load model + tokenizer ONLY ONCE
# -------------------------------------------------------
@st.cache_resource
def load_model_and_tokenizer():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Load tokenizer
    sp = spm.SentencePieceProcessor()
    sp.load("urdu_tokenizer.model")

    # Load model (Transformer class from Cell 2)
    model = Transformer(vocab_size=sp.vocab_size())
    model.load_state_dict(torch.load("best_masked_model.pt", map_location=device))
    model.to(device)
    model.eval()

    return model, sp, device

model, sp, device = load_model_and_tokenizer()

# -------------------------------------------------------
# Decoding Functions
# -------------------------------------------------------
def greedy_decode(model, src, sp, device, max_len=50):
    model.eval()
    src_mask = (src != 0).unsqueeze(1).unsqueeze(2)

    with torch.no_grad():
        enc_output = model.encoder_embedding(src) * (model.d_model ** 0.5)
        enc_output = model.positional_encoding(enc_output)
        for layer in model.encoder_layers:
            enc_output = layer(enc_output, src_mask)

    ys = torch.ones(1, 1, dtype=torch.long, device=device) * 2  # BOS token

    for _ in range(max_len - 1):
        tgt_mask = torch.tril(torch.ones((1, ys.size(1), ys.size(1)), device=device)).bool()
        out = model.decoder_embedding(ys) * (model.d_model ** 0.5)
        out = model.positional_encoding(out)
        dec_output = out
        for layer in model.decoder_layers:
            dec_output = layer(dec_output, enc_output, src_mask, tgt_mask)
        logits = model.fc_out(dec_output[:, -1])
        next_token = torch.argmax(logits, dim=-1).item()
        if next_token == 3:  # EOS token
            break
        ys = torch.cat([ys, torch.tensor([[next_token]], device=device)], dim=1)
    return sp.decode_ids(ys.squeeze().tolist()[1:])

def beam_search_decode(model, src, sp, device, beam_width=3, max_len=50):
    model.eval()
    src_mask = (src != 0).unsqueeze(1).unsqueeze(2)

    with torch.no_grad():
        enc_output = model.encoder_embedding(src) * (model.d_model ** 0.5)
        enc_output = model.positional_encoding(enc_output)
        for layer in model.encoder_layers:
            enc_output = layer(enc_output, src_mask)

    beams = [(torch.tensor([[2]], device=device), 0)]  # (sequence, score)
    for _ in range(max_len):
        new_beams = []
        for seq, score in beams:
            tgt_mask = torch.tril(torch.ones((1, seq.size(1), seq.size(1)), device=device)).bool()
            out = model.decoder_embedding(seq) * (model.d_model ** 0.5)
            out = model.positional_encoding(out)
            dec_output = out
            for layer in model.decoder_layers:
                dec_output = layer(dec_output, enc_output, src_mask, tgt_mask)
            logits = model.fc_out(dec_output[:, -1])
            probs = F.log_softmax(logits, dim=-1).squeeze(0)
            topk = torch.topk(probs, beam_width)
            for i in range(beam_width):
                next_tok = topk.indices[i].item()
                next_score = score + topk.values[i].item()
                new_seq = torch.cat([seq, torch.tensor([[next_tok]], device=device)], dim=1)
                new_beams.append((new_seq, next_score))
        beams = sorted(new_beams, key=lambda x: x[1], reverse=True)[:beam_width]
        if all(seq[0, -1].item() == 3 for seq, _ in beams):
            break
    best_seq = beams[0][0].squeeze().tolist()
    best_seq = [t for t in best_seq[1:] if t not in [0, 3]]
    return sp.decode_ids(best_seq)

def top_k_sampling(logits, k=50, temperature=0.9):
    logits = logits / temperature
    values, indices = torch.topk(logits, k)
    probs = F.softmax(values, dim=-1)
    return indices[torch.multinomial(probs, 1)].item()

def top_k_decode(model, src, sp, device, k=50, temperature=0.9, max_len=50):
    model.eval()
    src_mask = (src != 0).unsqueeze(1).unsqueeze(2)

    with torch.no_grad():
        enc_output = model.encoder_embedding(src) * (model.d_model ** 0.5)
        enc_output = model.positional_encoding(enc_output)
        for layer in model.encoder_layers:
            enc_output = layer(enc_output, src_mask)

    ys = torch.ones(1, 1, dtype=torch.long, device=device) * 2
    for _ in range(max_len - 1):
        tgt_mask = torch.tril(torch.ones((1, ys.size(1), ys.size(1)), device=device)).bool()
        out = model.decoder_embedding(ys) * (model.d_model ** 0.5)
        out = model.positional_encoding(out)
        dec_output = out
        for layer in model.decoder_layers:
            dec_output = layer(dec_output, enc_output, src_mask, tgt_mask)
        logits = model.fc_out(dec_output[:, -1])
        next_token = top_k_sampling(logits.squeeze(), k=k, temperature=temperature)
        if next_token == 3:
            break
        ys = torch.cat([ys, torch.tensor([[next_token]], device=device)], dim=1)
    return sp.decode_ids(ys.squeeze().tolist()[1:])

def generate_response(model, src_text, sp, device, decoding="Greedy", max_len=50):
    src_tokens = [2] + sp.encode_as_ids(src_text) + [3]
    src = torch.LongTensor([src_tokens]).to(device)
    if decoding == "Greedy":
        return greedy_decode(model, src, sp, device, max_len)
    elif decoding == "Beam Search":
        return beam_search_decode(model, src, sp, device, max_len=max_len)
    elif decoding == "Top-k Sampling":
        return top_k_decode(model, src, sp, device, max_len=max_len)

# -------------------------------------------------------
# Streamlit UI
# -------------------------------------------------------
st.set_page_config(page_title="Urdu Chatbot", layout="wide")
st.markdown("""
## 🇵🇰 Urdu Chatbot — Transformer Encoder-Decoder  
### **Decoding: Greedy | Beam Search | Top-k Sampling**  
Right-to-left Urdu text supported.
""")

# Maintain conversation history
if "history" not in st.session_state:
    st.session_state.history = []

col1, col2 = st.columns(2)

with col1:
    user_input = st.text_area(
        "📝 پیغام لکھیں",
        placeholder="یہاں لکھیں...",
        height=120
    )
    decoding_strategy = st.radio(
        "Decoding Strategy",
        ["Greedy", "Beam Search", "Top-k Sampling"],
        index=2
    )
    if st.button("Send ➤"):
        if user_input.strip():
            reply = generate_response(model, user_input, sp, device, decoding_strategy)
            st.session_state.history.append(f"آپ: {user_input}")
            st.session_state.history.append(f"بوٹ: {reply}")
            st.success(reply)
        else:
            st.error("براہ کرم کوئی پیغام لکھیں۔")

with col2:
    st.text_area(
        "گفتگو کا خلاصہ",
        value="\n".join(st.session_state.history),
        height=380
    )


2025-11-16 17:02:33.640 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-16 17:02:33.644 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-16 17:02:33.645 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-16 17:02:33.647 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-16 17:02:33.648 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-16 17:02:33.650 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-16 17:02:33.652 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-16 17:02:33.654 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar